## llm_zoomcamp module 2 
#### 02-vector-search


# COHORT 2

#### Q1. Embedding the query

In [1]:
from fastembed import TextEmbedding, SparseTextEmbedding

model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")
embeddings = list(model.embed(['I just discovered the course. Can I join now?']))

print(min(embeddings[0])) # What's the minimal value in this array?

-0.11726373885183883


In [2]:
import numpy as np
np.linalg.norm(embeddings)

1.0

In [3]:
q1_vector = embeddings[0]
q1_vector.dot(q1_vector)

1.0000000000000002

#### Q2. Cosine similarity with another vector

In [4]:
doc = 'Can I still join the course after the start date?'

In [5]:
emb = list(model.embed(doc))


In [6]:
q1_vector.dot(emb[0]) # What's the cosine similarity between the vector for the query and the vector for the document?

0.9008528895674548

#### Q3. Ranking by cosine

In [7]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [8]:
doc_texts = []
for doc in documents:
    doc_texts.append(doc.get('text'))


In [9]:
import numpy as np

doc_texts = [doc.get('text') for doc in documents]
doc_array = np.array(doc_texts).reshape(-1, 1)

In [10]:
emb_doc_texts = list(model.embed(doc_texts))

In [11]:
doc_array = np.array(emb_doc_texts).reshape(-1, 1)

In [19]:
max_similarity = 0
index_q3 = 0
for i, text in enumerate(emb_doc_texts):
    sim = text.dot(q1_vector)
    if sim > max_similarity:
        max_similarity = sim
        index_q3 = i
print(index_q3, '-> ', max_similarity) # What's the document index with the highest similarity? 

1 ->  0.8182378150042889


#### Q4. Ranking by cosine, version two

In [14]:
full_texts = [(doc.get('question') + ' ' + doc.get('text')) for doc in documents]
full_texts[0]

"Course - Can I still join the course after the start date? Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [15]:
full_texts[1]

'Course - Can I follow the course after it finishes? Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.'

In [16]:
emb_full_texts = list(model.embed(full_texts))
full_texts_arr = np.array(emb_full_texts).reshape(-1, 1)


In [17]:
# 'I just discovered the course. Can I join now?'
q1_vector.shape

(512,)

In [21]:
max_similarity4 = 0
index_q4 = 0
for i, text in enumerate(emb_full_texts):
    sim = text.dot(q1_vector)
    if sim > max_similarity4:
        max_similarity4 = sim
        index_q4 = i
print(index_q4, '-> ', max_similarity4)    # What's the highest scoring document?

0 ->  0.8514543236908068


###### Is it different from Q3? If yes, why?
There is difference between highest scoring document in Q3 (index 1) and Q4 (index 0) because in Q4 we are using vector from words from both question and answer, and in index 0 document question is almost the same like in query, so that vectors similarity becomes close to 1.

#### Q5. Selecting the embedding model

In [22]:
import json

mindim = 1000000
for model in TextEmbedding.list_supported_models():
    if model["dim"] < mindim:
        mindim = model["dim"]
print(mindim) # What's the smallest dimensionality for models in fastembed?

384


In [23]:
for model in TextEmbedding.list_supported_models():
    if model["dim"] == mindim:
        print(model["model"])

BAAI/bge-small-en
BAAI/bge-small-en-v1.5
snowflake/snowflake-arctic-embed-xs
snowflake/snowflake-arctic-embed-s
sentence-transformers/all-MiniLM-L6-v2
sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2


In [24]:
model_q5 = 'BAAI/bge-small-en'

####  Q6. Indexing with qdrant (2 points)

In [25]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [26]:
from qdrant_client import QdrantClient, models
qd_client = QdrantClient("http://localhost:6333")

In [29]:
# Define the collection name
collection_name = "zoomcamp-ml_only"


In [31]:
qd_client.delete_collection(collection_name=collection_name)

True

In [32]:
# Create the collection with specified vector parameters
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=mindim,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [33]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp'}

In [34]:
points = []

for i, doc in enumerate(documents):
    point = models.PointStruct(
        id=i,
        vector=models.Document(text=(doc.get('question') + ' ' + doc.get('text')), model=model_q5), #embed text locally 
    )
    points.append(point)



In [35]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [36]:
def search(query, limit=1):

    results = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_q5
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [37]:
# use the question from Q1 for querying the collection.
# What's the highest score in the results?
result = search('I just discovered the course. Can I join now?')

In [38]:
result

QueryResponse(points=[ScoredPoint(id=14, version=0, score=0.8703172, payload={}, vector=None, shard_key=None, order_value=None)])